<a href="https://colab.research.google.com/github/guillermoayllon/certificate_tensorflow/blob/main/fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn import model_selection

In [ ]:
(train, val, test), info = tfds.load('fashion_mnist', shuffle_files=True, as_supervised = True, split = ['train[:80%]','train[80%:]', 'test'], with_info = True)

 # EDA

In [ ]:
print(len(train))
print(len(val))
print(len(test))

48000
12000
10000


In [ ]:
train

<PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
x_train, y_train = [], []
for image, label in tfds.as_numpy(train):
    x_train.append(image)
    y_train.append(label)
 
x_val, y_val = [], []
for image, label in tfds.as_numpy(val):
    x_val.append(image)
    y_val.append(label)

In [ ]:
tfds.as_dataframe(train.take(5), info)

,image,label
0,,2 (Pullover)
1,,1 (Trouser)
2,,8 (Bag)
3,,4 (Coat)
4,,1 (Trouser)


In [ ]:
class_names = info.features["label"].names
n_classes = info.features['label'].num_classes
print(class_names, '\n', n_classes)

['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] 
 10


In [ ]:
np.unique(y_train, return_counts= True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([4763, 4800, 4810, 4794, 4855, 4791, 4798, 4766, 4799, 4824]))

In [ ]:
np.unique(y_val, return_counts= True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([1237, 1200, 1190, 1206, 1145, 1209, 1202, 1234, 1201, 1176]))

In [ ]:
np.unique(x_train)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

# Fully CNN

In [ ]:
train = train.shuffle(1000000).batch(32).prefetch(1)
val = val.shuffle(1000000).batch(32).prefetch(1)
def preprocess_FCNN(image, label):
    #image = tf.image.resize(image, [244,244])
    image = image/255
    return (image, label)

train = train.map(preprocess_FCNN)
val = val.map(preprocess_FCNN)
print(train)

<MapDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


Define a function to build the model:

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, 7, activation='relu', padding='same', input_shape=[28,28,1]))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Conv2D(128, 3, activation='relu', padding='same'))
model.add(keras.layers.Conv2D(128, 3, activation='relu', padding='same'))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Conv2D(256, 3, activation='relu', padding='same'))
model.add(keras.layers.Conv2D(256, 3, activation='relu', padding='same'))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Conv2D(10, 3, activation='relu', padding='valid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        3200      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 128)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 256)         2

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='Adam')

In [ ]:
model.fit(train, validation_data = val, epochs = 200, callbacks = [tf.keras.callbacks.EarlyStopping(patience=25)])

Epoch 1/200
 210/1500 [===>..........................] - ETA: 2:45 - loss: 2.7845

KeyboardInterrupt: 

# Experiments for later:

In [ ]:
def build_FCNN(hp, input_shape=16):
                #n_hidden = hp.Int("n_hidden", min_value = 1, max_value = 10, step = 1)
                #learning_rate = hp.Float("learning_rate", min_value = 3e-5, max_value = 3e-2)
                model = keras.models.Sequential()
                model.add(keras.layers.Conv2D(64, 7, activation='relu', padding='same', input_shape=[28,28,1]))
                model.add(keras.layers.MaxPooling2D(2))
                model.add(keras.layers.Conv2D(128, 3, activation='relu', padding='same'))
                model.add(keras.layers.Conv2D(128, 3, activation='relu', padding='same'))
                model.add(keras.layers.MaxPooling2D(2))
                model.add(keras.layers.Conv2D(256, 3, activation='relu', padding='same'))
                model.add(keras.layers.Conv2D(256, 3, activation='relu', padding='same'))
                model.add(keras.layers.MaxPooling2D(2))
                model.add(keras.layers.Conv2D(10, , activation='relu', padding='valid'))
                
                
                model.add(keras.layers.Dropout(rate=0.2))
                for layer in range(n_hidden):
                    model.add(keras.layers.Dense(n_neurons, activation="relu"))
                    model.add(keras.layers.Dropout(rate=0.2))
                model.add(keras.layers.Dense(1))
                opt = keras.optimizers.Adam(learning_rate)
                model.compile(loss='mean_squared_error',optimizer=opt)
                return model

Pass build_model to keras_tuner:

In [ ]:
#tuner = keras_tuner.tuners.BayesianOptimization(build_model, objective="val_loss", max_trials = 50, overwrite = True)
tuner = keras_tuner.tuners.RandomSearch(build_model, objective="val_loss", max_trials=200, executions_per_trial=1, overwrite = True)

In [ ]:
tuner.search(train[:,:-2],train[:,-2], epochs = 200,validation_data = (val[:,:-2],val[:,-2]), callbacks=[keras.callbacks.EarlyStopping(patience=25)])

Get parameters and models:

In [ ]:
best_hp = tuner.get_best_hyperparameters(200)
best_models = tuner.get_best_models(200)

In [ ]:
def build_FCNN()